In [ ]:
from attention_project_utils import *
from suite2p import run_s2p
import imagej
import matplotlib.pyplot as plt
import os
import numpy as np
import scyjava_config
%matplotlib notebook

# Path to your Fiji installation that includes
fiji_path = 'fiji'
#fiji_path = 'C:\\Users\\Semmelhack Lab\\Downloads\\fiji-win64\\Fiji'
# Initialize Fiji
scyjava_config.add_options('-Xmx8g')
ij = imagej.init(fiji_path)
ij.getVersion()
ij.ui().showUI()

## Select directory containing `.h5` and `.nd2` data

In [ ]:
fish_dir = "D:/BISWADEEP_DATA/"

In [ ]:
nd2_files = list(filter(lambda x: "wholebrain" not in x, list_files(fish_dir, extension='nd2')))

processed_dir = fish_dir + '/processed'
avi_dir = processed_dir + '/avi'
raw_tif_dir = processed_dir + '/raw tif'
combined_tif_dir = processed_dir + '/combined tif/'
intensity_dir = processed_dir + '/intensity'
registered_tif_dir = processed_dir + '/registered tif'
comb_registered_tif_dir = combined_tif_dir + '/combined_registered'
neural_dir = processed_dir + '/neural'
behavior_dir = processed_dir + '/behavior'

for directory in [processed_dir, avi_dir, raw_tif_dir, combined_tif_dir,
                  intensity_dir, registered_tif_dir, neural_dir, behavior_dir, comb_registered_tif_dir]:
    create_directory(directory)
    
raw_tif_files = list(map(lambda x: x.replace('nd2', 'tif').replace(fish_dir, raw_tif_dir), nd2_files))

## Macro for converting `.nd2` image data to `.tif` files

In [ ]:
nd2_to_tif = """
#@ String nd2_path
#@ String save_path
setBatchMode(true);
//run("Memory & Threads...", "maximum=12143 parallel=8 run");
run("Bio-Formats Importer", "open=[" + nd2_path + "] autoscale color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT");
run("Rotate 90 Degrees Left");
run("8-bit");
saveAs("Tiff", save_path);
run("Close All");
"""

In [ ]:

for nd2_file, tif_file  in zip(nd2_files, raw_tif_files):
    args = {
        'nd2_path': nd2_file,
        'save_path': tif_file
    }
    ij.py.run_script('ijm', nd2_to_tif, args)

## Macro for combining multiple `.tif` files into one `.tif` file

In [ ]:
combine_tif = """
#@ String[] tif_files
#@ String save_directory
#@output String slices
setBatchMode(true);
open(tif_files[0]);
image1 = getTitle();
slices = toString(nSlices);

for (i = 1; i < tif_files.length; ++i) {
    open(tif_files[i]);
    image2 = getTitle();
    slices = slices + " " + toString(nSlices);
    run("Concatenate...", "open image1=" + image1 + " image2="+ image2);
    image1 = getTitle();
}
saveAs("Tiff", save_directory + "combined.tif");
run("Close All");
"""

In [ ]:
args = {
    'tif_files': raw_tif_files,
    'save_directory': combined_tif_dir + '/'
}

slices = ij.py.run_script('ijm', combine_tif, args).getOutput('slices')
slices = list(map(lambda x: int(x), slices.split(" ")))
print(slices)

## Run suite2p

In [ ]:
ops = run_s2p.default_ops()
ops['batch_size'] = 200
ops['roidetect'] = False
ops['reg_tif'] = True
ops['save_folder'] =combined_tif_dir
ops['reg_tif'] = True

db = {
      'h5py': [],
      'h5py_key': 'data',
      'look_one_level_down': False,
      'data_path': [combined_tif_dir],                     
      'subfolders': [], 'reg_tif': True
    }

In [ ]:
opsEnd=run_s2p.run_s2p(ops=ops,db=db)

In [ ]:
## MACRO FOR COMBINING THE REGISTERED TIFS

In [ ]:
combine_tif = """
#@ String[] tif_files
#@ String save_directory
#@output String slices
setBatchMode(true);
open(tif_files[0]);
image1 = getTitle();
slices = toString(nSlices);

for (i = 1; i < tif_files.length; ++i) {
    open(tif_files[i]);
    image2 = getTitle();
    slices = slices + " " + toString(nSlices);
    run("Concatenate...", "open image1=" + image1 + " image2="+ image2);
    image1 = getTitle();
}
saveAs("Tiff", save_directory + "combined_reg.tif");
run("Close All");
"""

In [ ]:
args = {
    'tif_files': list_files(combined_tif_dir +'/plane0/reg_tif'),
    'save_directory': comb_registered_tif_dir + '/'
}

slices = ij.py.run_script('ijm', combine_tif, args).getOutput('slices')
slices = list(map(lambda x: int(x), slices.split(" ")))
print(slices)

## Macro for watershed segmentation

In [ ]:
segmentation = """
#@ String image_path

open(image_path);
window_name = getTitle();
output_directory = replace(image_path, window_name, "");
run("Z Project...", "projection=[Average Intensity]");
selectWindow(window_name);
run("Close");
avg = getTitle();
run("8-bit");
setTool("rectangle");
waitForUser("Pause", "Draw midbox"); // Ask for input ROI
saveAs("XY Coordinates", output_directory + "Midline.txt");
run("Select None");
setTool("polygon");
waitForUser("Pause", "Draw a mask");
run("Create Mask");
saveAs("Tiff", output_directory + "Mask.tif");
imageCalculator("AND create stack", avg , "Mask.tif");
run("Enhance Contrast...", "saturated=0.3 normalize");
run("Median...", "radius=1 stack");
run("Morphological Filters", "operation=[White Top Hat] element=Disk radius=3");
run("Invert");
run("Watershed Segmentation");
waitForUser("Pause", "Check results");
saveAs("Tiff", output_directory + "label.tif");
run("Analyze Regions", "area perimeter circularity centroid ");
saveAs("Results", output_directory + "label-Morphometry.csv");
run("Close All");
"""

In [ ]:
registered_tif = comb_registered_tif_dir + "/combined_reg.tif"

args = {
    'image_path': registered_tif,
}
ij.py.run_script('ijm', segmentation, args)

## Macro for separating registered combined `.tif` imaging data

In [ ]:
separate = """
#@ String combined_tif_path
#@ String save_path
#@ String nfiles
#@ String nfiles2

setBatchMode(true);
open(combined_tif_path);
main_window = getTitle();
folder = replace(combined_tif_path, main_window, "");
save_folder = folder + "results\\";

File.makeDirectory(replace(combined_tif_path, main_window, "results"));
length_per_slice = toString(round(nSlices/nfiles));
print(length_per_slice);
for (i = 0; i < nfiles; ++i) {
    if (i < nfiles2) {
        run("Make Substack...", "delete slices=1-" + length_per_slice);
        curr_win = getTitle();
    }
    run("8-bit");
    saveAs("Tiff", save_path + "T" + toString(i+1));
    selectWindow("T" + toString(i+1)  + ".tif");
    run("Close");
}
run("Close All");

"""

In [ ]:
nfiles = 6 # assuming the number of images per file are uniform
args = {
    'combined_tif_path':  comb_registered_tif_dir + "/combined_reg.tif",
    'save_path': registered_tif_dir + '/',
    'nfiles': nfiles,
    'nfiles2': nfiles-1
}
ij.py.run_script('ijm', separate, args)

## Macro for getting the intensities of ROIs 

In [ ]:
get_intensity = """
setBatchMode(true)
#@ String label
#@ String save_folder
#@ String[] file_list
setBatchMode(true);
for (i = 0; i < file_list.length; ++i) {
    temp = split(file_list[i], "/");
    getInt(file_list[i], label, save_folder + replace(temp[temp.length - 1], ".tif", "/"));
}

function getInt(image, label, save_folder) {
    if (!File.exists(save_folder)) {
        File.makeDirectory(save_folder);
    }
    open(label);
    label_window = replace(getTitle(), ".tif", "");
    open(image);
    image = getTitle();
    run("8-bit");
    for(i = 1; i <= nSlices; ++i) {
        setSlice(i); // start from first frame
        run("Duplicate...", "use"); // isolate the frame
        current_slice = getTitle();
        print(current_slice);
        curr = replace(current_slice, ".tif", "");
        //print(curr);
        run("Intensity Measurements 2D/3D", "input=" + curr + " labels=" + label_window +  " mean");
        saveAs("Results", save_folder + "int_t" + toString(i) + ".csv");
        run("Close");
        selectWindow(current_slice);
        run("Close");
    }
}
"""

In [ ]:
label = comb_registered_tif_dir + '/label.tif'

args = {
    'label': label,
    'save_folder': intensity_dir + '/',
    'file_list': list_files(registered_tif_dir),
}
ij.py.run_script('ijm', get_intensity, args)

## Combine and process `.csv` files

In [ ]:
morphology = pd.read_csv( comb_registered_tif_dir+ '/label-Morphometry.csv')
neurons = np.array(morphology[(morphology.Area < 300) & (morphology.Area > 10) & (morphology.Circularity > 0.5) & (morphology.Circularity < 1.5)].Label)

for trial_dir in list_directories(intensity_dir):
    split = trial_dir.split('/')[-1].replace('-', '_').replace('.', '_').split('_')
    fish_trial = split[0]
    df = pd.concat([pd.read_csv(int_file).set_index('Label').rename(
                   columns={'Mean': int(int_file.split('t')[-1].split('.')[0]) - 1})
                   for int_file in list_files(trial_dir)], axis=1)

    df = df.reindex(sorted(df.columns), axis=1).transpose()
    new_df = df[neurons].transpose()
    new_df.to_csv(neural_dir + '/' + fish_trial + '.csv')

In [ ]:
def view_roi(label, vis_tif, selected_labels):
    '''
    function to visualise the neurons in Fiji
    label is the image containing the segmented ROIs
    vis_tif is the timelapse image you want to overlap the ROIs with
    selected_labels are the index number of ROIs you want to visualize
    '''
    ROI_viewer = """
    #@ String label
    #@ String reg_tif
    #@ String labels

    open(label);
    label = getTitle();
    run("Select Label(s)", "label(s)=" + labels);
    run("Labels To RGB", "colormap=Spectrum background=Black shuffle");
    run("Morphological Filters", "operation=[Internal Gradient] element=Disk radius=0.5");
    new_label = getTitle();
    selectWindow(label);
    run("Close");
    open(reg_tif);
    img = getTitle();
    run("RGB Color");
    imageCalculator("Add create stack", img, new_label);

    selectWindow("label-keepLabels");
    run("Close");
    selectWindow("label-keepLabels-rgb");
    run("Close");

    selectWindow(img);
    run("Z Project...", "projection=[Average Intensity]");
    avg = getTitle();
    selectWindow(img);
    run("Close");
    imageCalculator("Add create", avg, new_label);
    lbl_avg = getTitle();
    run("Concatenate...", "open image1=["+ avg +"] image2=[" + lbl_avg + "] image3=[-- None --]");
    selectWindow("label-keepLabels-rgb-Internal Gradient");
    run("Close");
    """


    args = {
        'label': label,
        'reg_tif': vis_tif,
        'labels': selected_labels,
    }

    ij.py.run_script('ijm',ROI_viewer, args)

In [ ]:
label = comb_registered_tif_dir + '/label.tif'
reg_tif =  registered_tif_dir + '/T1.tif'
selected_labels = list(pd.read_csv(neural_dir+ '/T1.csv').Label)

view_roi(label, reg_tif, selected_labels)